<a href="https://colab.research.google.com/github/Shreyas-pandith/NLP_PROJECTS/blob/dev/NLP_PROJECTS%20/Sentiment_Analysis_LSTM/lstm_arch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import nltk
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment-analysis-dataset


In [ ]:
!ls /kaggle/input/sentiment-analysis-dataset

test.csv			train.csv
testdata.manual.2009.06.14.csv	training.1600000.processed.noemoticon.csv


In [ ]:
import pandas as pd

# Load CSV file into a DataFrame
df = pd.read_csv('/kaggle/input/sentiment-analysis-dataset/train.csv',encoding='latin1')
#print(df.head())
df=df[['selected_text','sentiment']]
print(df.head())

                         selected_text sentiment
0  I`d have responded, if I were going   neutral
1                             Sooo SAD  negative
2                          bullying me  negative
3                       leave me alone  negative
4                        Sons of ****,  negative


In [ ]:
nltk.download('punkt',force=True)
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
stw = list(stopwords.words('english'))
print(stw)
stw=[]
from nltk import word_tokenize

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
x_train = df['selected_text']
y_train = df['sentiment']
print(len(x_train))
print(len(y_train))
print(x_train[:5])
print(y_train[:5])

27481
27481
0    I`d have responded, if I were going
1                               Sooo SAD
2                            bullying me
3                         leave me alone
4                          Sons of ****,
Name: selected_text, dtype: object
0     neutral
1    negative
2    negative
3    negative
4    negative
Name: sentiment, dtype: object


In [ ]:
def remove_stopwords(x):
  tokenized_sentence = word_tokenize(str(x).lower())
  filtered_tokens = [token  for token in tokenized_sentence if token not in stw and token.isalnum()]
  return filtered_tokens
sentence_l = x_train.apply(remove_stopwords)
y_train = y_train.map({'positive':2,'negative':0,'neutral':1})
print(sentence_l[:5])
print(y_train[:5])


0    [i, d, have, responded, if, i, were, going]
1                                    [sooo, sad]
2                                 [bullying, me]
3                             [leave, me, alone]
4                                     [sons, of]
Name: selected_text, dtype: object
0    1
1    0
2    0
3    0
4    0
Name: sentiment, dtype: int64


In [ ]:
vocab ={}
for sentence in sentence_l:
  for word in sentence:
    if word in vocab:
      vocab[word]+=1
    else:
      vocab[word]=1
print(len(vocab))

for word in list(vocab.keys()):
  if vocab[word]<2:
    del vocab[word]
print(len(vocab))

index = 1
reverse_vocab = {}
for word in list(vocab.keys()):
  vocab[word] = index
  reverse_vocab[index] = word
  index+=1

vocab['<unk>']=index
reverse_vocab[index] = '<unk>'
index+=1
vocab['<pad>']=index
reverse_vocab[index] = '<pad>'

print(vocab)
print(reverse_vocab)



16454
6786
{'i': 1, 'd': 2, 'have': 3, 'if': 4, 'were': 5, 'going': 6, 'sooo': 7, 'sad': 8, 'me': 9, 'leave': 10, 'alone': 11, 'sons': 12, 'of': 13, 'http': 14, 'some': 15, 'plugging': 16, 'for': 17, 'the': 18, 'best': 19, 'rangers': 20, 'forum': 21, 'on': 22, 'earth': 23, 'fun': 24, 'soooo': 25, 'high': 26, 'both': 27, 'you': 28, 'wow': 29, 'u': 30, 'just': 31, 'became': 32, 'cooler': 33, 'as': 34, 'much': 35, 'love': 36, 'to': 37, 'be': 38, 'hopeful': 39, 'reckon': 40, 'chances': 41, 'are': 42, 'minimal': 43, 'm': 44, 'never': 45, 'gon': 46, 'na': 47, 'get': 48, 'my': 49, 'cake': 50, 'and': 51, 'stuff': 52, 'like': 53, 'lost': 54, 'test': 55, 'from': 56, 'lg': 57, 'uh': 58, 'oh': 59, 'am': 60, 'sunburned': 61, 'sigh': 62, 'sick': 63, 'hes': 64, 'not': 65, 'that': 66, 'into': 67, 'so': 68, 'sorry': 69, 'hope': 70, 'find': 71, 'her': 72, 'soon': 73, '3': 74, 'interesting': 75, 'is': 76, 'cleaning': 77, 'house': 78, 'family': 79, 'who': 80, 'comming': 81, 'later': 82, 'today': 83, 'got'

In [ ]:
def tokenize_words(word_list):
  for index,word in enumerate(word_list,0):
    if word in vocab:
      word_list[index] = vocab[word]
    else:
      word_list[index] = vocab['<unk>']
  return word_list

In [ ]:
sentence_l = sentence_l.apply(tokenize_words)
print(sentence_l[:5])
print(len(sentence_l))
print(len(y_train))
non_empty_mask = sentence_l.apply(len) > 0
sentence_l = sentence_l[non_empty_mask].reset_index(drop=True)
y_train = pd.Series(y_train)
y_train = y_train[non_empty_mask].reset_index(drop=True)
print(len(sentence_l))
print(len(y_train))

0    [1, 2, 3, 6787, 4, 1, 5, 6]
1                         [7, 8]
2                      [6787, 9]
3                    [10, 9, 11]
4                       [12, 13]
Name: selected_text, dtype: object
27481
27481
27320
27320


In [24]:


class RNN(nn.Module):
  def __init__(self,emb_dim,h_dim):
    super().__init__()
    print('RNN init')
    self.emd_dim = emb_dim
    self.h_dim = h_dim
    self.Wi = nn.Linear(emb_dim,h_dim,bias=True)
    self.Wh = nn.Linear(h_dim,h_dim,bias=True)

    self.norm = nn.LayerNorm(h_dim)
    self.act = nn.Tanh()
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  def forward(self,x):
    batch_size , sequence_length , emb_dim = x.size()

    h = torch.zeros(batch_size,self.h_dim).to(self.device)
    output = torch.zeros(batch_size,sequence_length,self.h_dim).to(self.device)
    for i in range(0,sequence_length):
      #print('in loop ',i)
      inp = x[:,i,:]
      #print('inp shape ',inp.shape)

      h =  self.act(self.norm (self.Wi(inp)+self.Wh(h)))
      output[:,i,:] =h

    return output

class Sequence_model(nn.Module):
  def __init__(self,V,embd_dim,h_dim):
    super().__init__()
    print("init Sequence model")

    self.h_dim = h_dim
    self.embd_dim = embd_dim
    self.embedding_layer = nn.Embedding(V,self.embd_dim)
    self.rnn_layer =  RNN(self.embd_dim,self.h_dim)
    self.dropout = nn.Dropout(0.5)
    self.dropout2 = nn.Dropout(0.5)
    self.rnn_layer2 =  RNN(self.h_dim,self.h_dim)
    self.fc = nn.Linear(self.h_dim,3)
    #self.act = nn.Softmax(dim=1)

  def forward(self,x):
    #print('in sequence forward ',x.shape)

    x = self.embedding_layer(x)


    #print('embedded shape ',x.shape)

    out = self.rnn_layer(x)
    out = self.dropout(out)
    out = self.rnn_layer2(out)
    out = self.dropout2(out)
    #print('rnn out shape ',out.shape)
    out = self.fc(out[:,-1,:])
    #print('fc out shape ',out.shape)
    #out = self.act(out)
    #print('act out shape ',out.shape)
    return out



In [45]:
class RNN(nn.Module):
  def __init__(self,emb_dim,h_dim):
    super().__init__()
    print('RNN init')
    self.emd_dim = emb_dim
    self.h_dim = h_dim
    self.Wi = nn.Linear(emb_dim,h_dim,bias=True)
    self.Wh = nn.Linear(h_dim,h_dim,bias=True)

    self.norm = nn.LayerNorm(h_dim)
    self.act = nn.Tanh()
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  def forward(self,x):
    batch_size , sequence_length , emb_dim = x.size()

    h = torch.zeros(batch_size,self.h_dim).to(self.device)
    output = torch.zeros(batch_size,sequence_length,self.h_dim).to(self.device)
    for i in range(0,sequence_length):
      #print('in loop ',i)
      inp = x[:,i,:]
      #print('inp shape ',inp.shape)

      h =  self.act(self.norm (self.Wi(inp)+self.Wh(h)))
      output[:,i,:] =h

    return output

class Sequence_model(nn.Module):
  def __init__(self,V,embd_dim,h_dim):
    super().__init__()
    print("init Sequence model")

    self.h_dim = h_dim
    self.embd_dim = embd_dim
    self.embedding_layer = nn.Embedding(V,self.embd_dim)
    self.rnn_layer =  RNN(self.embd_dim,self.h_dim)
    self.dropout = nn.Dropout(0.5)
    self.dropout2 = nn.Dropout(0.5)
    self.rnn_layer2 =  RNN(self.h_dim,self.h_dim)
    self.fc = nn.Linear(self.h_dim,3)
    #self.act = nn.Softmax(dim=1)

  def forward(self,x):
    #print('in sequence forward ',x.shape)

    x = self.embedding_layer(x)


    #print('embedded shape ',x.shape)

    out = self.rnn_layer(x)
    out = self.dropout(out)
    out = self.rnn_layer2(out)
    out = self.dropout2(out)
    #print('rnn out shape ',out.shape)
    out = self.fc(out[:,-1,:])
    #print('fc out shape ',out.shape)
    #out = self.act(out)
    #print('act out shape ',out.shape)
    return out

In [ ]:
print( [{name ,p.shape} for name,p in lstm_model.named_parameters()])

In [ ]:
from torch.utils.data import Dataset
class sentence_sim_dataset(Dataset):
  def __init__(self,sentence_l,y_train,max_len,vocab):
    self.sentence_l = sentence_l
    self.y_train = y_train
    self.max_len = max_len

  def __len__(self):
    print(len(self.sentence_l))
    return len(self.sentence_l)

  def __getitem__(self,index):
    sentence = self.sentence_l[index]
    if(len(sentence)<self.max_len):
      sentence = sentence + [vocab['<pad>']]*(self.max_len-len(sentence))
    else:
      sentence = sentence[:self.max_len]

    #print(sentence.shape)
    sentence = torch.tensor(sentence,dtype=torch.long)
    label = torch.tensor(self.y_train[index],dtype=torch.long)



    return sentence,label

In [ ]:
dataset = sentence_sim_dataset(sentence_l,y_train,12,vocab)
sentence_lengths={}
for s in sentence_l:
  if(len(s) in sentence_lengths):
    sentence_lengths[len(s)]+=1
  else:
    sentence_lengths[len(s)]=1
print(sentence_lengths)
dataloader = torch.utils.data.DataLoader(dataset,batch_size=32,shuffle=False)


{8: 1032, 2: 2476, 3: 2065, 11: 681, 1: 6905, 5: 1610, 24: 296, 6: 1365, 14: 504, 12: 642, 18: 400, 10: 804, 22: 318, 4: 2092, 20: 375, 27: 143, 17: 416, 9: 847, 13: 599, 19: 370, 16: 460, 26: 190, 28: 96, 7: 1153, 31: 18, 23: 280, 21: 349, 29: 58, 15: 481, 25: 259, 30: 27, 33: 5, 32: 4}


In [25]:
def get_back_sentence(tokens):

  tokens = [reverse_vocab[t] for t in tokens]
  return tokens

count = 0
for item_X , item_Y in dataset:
  if(count %1000 ==0):
    print(get_back_sentence(item_X.tolist()))
    print(item_Y)
  count+=1


['i', 'd', 'have', '<unk>', 'if', 'i', 'were', 'going', '<pad>', '<pad>', '<pad>', '<pad>']
tensor(1)
['i', 'hope', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
tensor(2)
['i', 'miss', 'having', 'an', 'it', '<unk>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
tensor(0)
['good', 'weekend', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
tensor(2)
['awe', 'i', 'am', 'sorry', 'probably', 'true', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
tensor(2)
['worth', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
tensor(2)
['i', 'wan', 'na', 'go', 'there', 'where', 'you', 'go', 'i', 'wan', 'na', 'find']
tensor(1)
['to', 'enjoy', 'a', 'lovely', 'bbq', 'with', 'the', 'family', 'loving', 'the', 'may', 'day']
tensor(2)
['is', 'go', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
tensor(2)
['so', 'afraid'

KeyError: 27320

In [36]:
lstm_model = Sequence_model(len(vocab)+1,128,512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lstm_model.to(device)


from tqdm import tqdm
def loss_func(out,target):
  return nn.CrossEntropyLoss()(out,target)
ep=0
optim = torch.optim.Adam(lstm_model.parameters(),lr=0.001)
for ep in range(0,15):
  print("epoch ",ep)
  pbar = tqdm(dataloader, leave=True, desc=f"Epoch {ep}")
  for batch in pbar:
    lstm_model.zero_grad()
    item_x , item_y = batch

    item_x = item_x.to(device)
    item_y = item_y.to(device)
    #print(item_x.shape)
    #print(item_y.shape)
    out = lstm_model(item_x)
    #print(out.shape)
    #print(item_y.shape)
    loss = loss_func(out,item_y)
    loss.backward()
    optim.step()
    pbar.set_postfix(loss=loss.item())
  torch.save({
    'epoch': ep,
    'model_state_dict': lstm_model.state_dict(),
    'optimizer_state_dict': optim.state_dict(),
    'loss': loss.item,
  }, '/content/drive/MyDrive/nlp_learnings/lstm_project/rnn_checkpoints/'+str(ep)+'.pth')





init Sequence model
RNN init
epoch  0
27320


Epoch 0:   1%|          | 6/854 [00:00<00:14, 58.53it/s, loss=1.07]

27320


Epoch 0: 100%|██████████| 854/854 [00:11<00:00, 72.93it/s, loss=0.643]


epoch  1
27320


...

Epoch 13: 100%|██████████| 854/854 [00:12<00:00, 68.63it/s, loss=0.159]


epoch  14
27320


Epoch 14:   1%|          | 8/854 [00:00<00:11, 72.81it/s, loss=0.237]

27320


Epoch 14: 100%|██████████| 854/854 [00:12<00:00, 68.43it/s, loss=0.14]


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
%cd /content/drive/MyDrive/nlp_learnings/lstm_project/
!ls

/content/drive/MyDrive/nlp_learnings/lstm_project
lstm_arch.ipynb  rnn_checkpoints


In [ ]:
!ls

lstm_arch.ipynb  rnn_checkpoints


In [44]:
def convert_to_token_test(x):
  tokenized_sentence = word_tokenize(str(x).lower())
  print(tokenized_sentence)
  filtered_tokens = [token  for token in tokenized_sentence if token not in stw and token.isalnum()]
  print(filtered_tokens)
  tokenized_words= tokenize_words(filtered_tokens)
  if(len(tokenized_words)<12):
      tokenized_words = tokenized_words + [vocab['<pad>']]*(12-len(tokenized_sentence))
  else:
      tokenized_words = tokenized_words[:12]

    #print(sentence.shape)
  tokenized_words = torch.tensor(tokenized_words,dtype=torch.long)
  return tokenized_words


#test_inputs = ["I absolutely loved this movie, it was fantastic!","The service was excellent and the staff were friendly.","What a wonderful day it has been!","I'm feeling very optimistic about the future.","The food was delicious and the ambiance was perfect."]
#test_inputs =["This is the worst experience I've ever had.","I hated every minute of that film.","The product broke after just one use.","I am very disappointed with the customer service.","Nothing went right today; it was a terrible day."]
#test_inputs=["I went to the store to buy some groceries.","The meeting was scheduled for 3 PM.","She walked her dog in the park.","It rained in the afternoon but cleared up later.","The phone has a 6.5-inch screen and a decent battery life."]
test_inputs = ["he was a nice guy","movie was pretty boring","how on earth can she say that","cannot belive its a miracle","happy to say you have passed"]
with torch.no_grad():
  lstm_model.eval()
  for ti in test_inputs:
    testing = convert_to_token_test(ti)
    print(testing)
    out = lstm_model(testing.unsqueeze(0).to(device))
    out = torch.argmax(out,dim=1)
    print(out)

['he', 'was', 'a', 'nice', 'guy']
['he', 'was', 'a', 'nice', 'guy']
tensor([ 720,   90,  158,  430, 1945, 6788, 6788, 6788, 6788, 6788, 6788, 6788])
tensor([2], device='cuda:0')
['movie', 'was', 'pretty', 'boring']
['movie', 'was', 'pretty', 'boring']
tensor([1273,   90,  374,  496, 6788, 6788, 6788, 6788, 6788, 6788, 6788, 6788])
tensor([0], device='cuda:0')
['how', 'on', 'earth', 'can', 'she', 'say', 'that']
['how', 'on', 'earth', 'can', 'she', 'say', 'that']
tensor([ 196,   22,   23,  279,  286,  300,   66, 6788, 6788, 6788, 6788, 6788])
tensor([1], device='cuda:0')
['can', 'not', 'belive', 'its', 'a', 'miracle']
['can', 'not', 'belive', 'its', 'a', 'miracle']
tensor([ 279,   65, 6787,  138,  158, 6787, 6788, 6788, 6788, 6788, 6788, 6788])
tensor([1], device='cuda:0')
['happy', 'to', 'say', 'you', 'have', 'passed']
['happy', 'to', 'say', 'you', 'have', 'passed']
tensor([ 160,   37,  300,   28,    3,  907, 6788, 6788, 6788, 6788, 6788, 6788])
tensor([2], device='cuda:0')
